# Retrieve Graph

#### Node Dictionaries
+ track_dict
+ playlist_dict

#### Edge Dictionaries
+ shared_album_edges
+ shared_artist_edges
+ shared_genre_edges
+ cosine_similarity_edges
+ contains_edges

In [1]:
from py2neo import Graph, Node, Relationship
import numpy as np
from collections import defaultdict

uri = "..."
username = "..."
password = "..."
graph = Graph(uri, auth=(username, password))

# Retrieve track nodes and their properties
track_query = '''
MATCH (t:Track)
RETURN t.acousticness, t.album_id, t.artist_ids, t.danceability, t.duration_ms, t.energy, t.explicit,
       t.genre, t.id, t.key, t.liveness, t.loudness, t.mode, t.popularity, t.speechiness, t.tempo, t.valence
'''

track_result = graph.run(track_query).data()

# Create dictionary for tracks
track_dict = {row['t.id']: {
    'acousticness': row['t.acousticness'],
    'album_id': row['t.album_id'],
    'artist_ids': row['t.artist_ids'],
    'danceability': row['t.danceability'],
    'duration_ms': row['t.duration_ms'],
    'energy': row['t.energy'],
    'explicit': row['t.explicit'],
    'genre': row['t.genre'],
    'key': row['t.key'],
    'liveness': row['t.liveness'],
    'loudness': row['t.loudness'],
    'mode': row['t.mode'],
    'popularity': row['t.popularity'],
    'speechiness': row['t.speechiness'],
    'tempo': row['t.tempo'],
    'valence': row['t.valence'],
} for row in track_result}

# Retrieve playlist nodes and their properties
playlist_query = '''
MATCH (p:Playlist)
RETURN p.playlist_id, p.tracklist
'''

playlist_result = graph.run(playlist_query).data()

# Create dictionary playlists
playlist_dict = {row['p.playlist_id']: row['p.tracklist'] for row in playlist_result}

# Create defaultdicts for relationships
shared_album_edges = defaultdict(list)
shared_artist_edges = defaultdict(list)
shared_genre_edges = defaultdict(list)
cosine_similarity_edges = defaultdict(list)
contains_edges = defaultdict(list)

# Retrieve SHARED_ALBUM relationships
shared_album_query = '''
MATCH (t1:Track)-[:SHARED_ALBUM]->(t2:Track)
RETURN t1.id, t2.id
'''
shared_album_result = graph.run(shared_album_query).data()

for row in shared_album_result:
    track1_id, track2_id = row['t1.id'], row['t2.id']
    shared_album_edges[track1_id].append(track2_id)
    shared_album_edges[track2_id].append(track1_id)

# Retrieve SHARED_ARTIST relationships
shared_artist_query = '''
MATCH (t1:Track)-[:SHARED_ARTIST]->(t2:Track)
RETURN t1.id, t2.id
'''
shared_artist_result = graph.run(shared_artist_query).data()

for row in shared_artist_result:
    track1_id, track2_id = row['t1.id'], row['t2.id']
    shared_artist_edges[track1_id].append(track2_id)
    shared_artist_edges[track2_id].append(track1_id)

# Retrieve SHARED_GENRE relationships
shared_genre_query = '''
MATCH (t1:Track)-[:SHARED_GENRE]->(t2:Track)
RETURN t1.id, t2.id
'''
shared_genre_result = graph.run(shared_genre_query).data()

for row in shared_genre_result:
    track1_id, track2_id = row['t1.id'], row['t2.id']
    shared_genre_edges[track1_id].append(track2_id)
    shared_genre_edges[track2_id].append(track1_id)
    
# Retrieve COSINE_SIMILARITY relationships
cosine_similarity_query = '''
MATCH (t1:Track)-[r:COSINE_SIMILARITY]->(t2:Track)
RETURN t1.id, t2.id, r.value
'''
cosine_similarity_result = graph.run(cosine_similarity_query).data()

for row in cosine_similarity_result:
    track1_id, track2_id, similarity = row['t1.id'], row['t2.id'], row['r.value']
    cosine_similarity_edges[track1_id].append(track2_id)
    cosine_similarity_edges[track2_id].append(track1_id)

# Retrieve CONTAINS relationships
contains_query = '''
MATCH (p:Playlist)-[:CONTAINS]->(t:Track)
RETURN p.playlist_id, t.id
'''
contains_result = graph.run(contains_query).data()

for row in contains_result:
    playlist_id, track_id = row['p.playlist_id'], row['t.id']
    contains_edges[playlist_id].append(track_id)

# Creating Adjacency Matrices

In [2]:
from scipy.sparse import lil_matrix

# Get unique track IDs
unique_track_ids = list(track_dict.keys())

# Create a track ID to index mapping
track_id_to_index = {track_id: index for index, track_id in enumerate(unique_track_ids)}

#print(track_id_to_index)

# Get unique playlist IDs
unique_playlist_ids = list(playlist_dict.keys())

# Create a playlist ID to index mapping
playlist_id_to_index = {playlist_id: index for index, playlist_id in enumerate(unique_playlist_ids)}

#print(playlist_id_to_index)

# Create adjacency matrices for each relationship
n_tracks = len(unique_track_ids)
shared_album_adj = lil_matrix((n_tracks, n_tracks), dtype=np.float32)
shared_artist_adj = lil_matrix((n_tracks, n_tracks), dtype=np.float32)
shared_genre_adj = lil_matrix((n_tracks, n_tracks), dtype=np.float32)
cosine_similarity_adj = lil_matrix((n_tracks, n_tracks), dtype=np.float32)

# Fill in the shared_album_adj matrix
for track_id, connected_track_ids in shared_album_edges.items():
    track_index = track_id_to_index[track_id]
    for connected_track_id in connected_track_ids:
        connected_track_index = track_id_to_index[connected_track_id]
        shared_album_adj[track_index, connected_track_index] = .25

# Fill in the shared_artist_adj matrix
for track_id, connected_track_ids in shared_artist_edges.items():
    track_index = track_id_to_index[track_id]
    for connected_track_id in connected_track_ids:
        connected_track_index = track_id_to_index[connected_track_id]
        shared_artist_adj[track_index, connected_track_index] = .25

# Fill in the shared_genre_adj matrix
for track_id, connected_track_ids in shared_genre_edges.items():
    track_index = track_id_to_index[track_id]
    for connected_track_id in connected_track_ids:
        connected_track_index = track_id_to_index[connected_track_id]
        shared_genre_adj[track_index, connected_track_index] = .5

# Fill in the cosine_similarity_adj matrix
for track_id, connected_track_tuples in cosine_similarity_edges.items():
    track_index = track_id_to_index[track_id]
    for connected_track_id in connected_track_tuples:
        connected_track_index = track_id_to_index[connected_track_id]
        cosine_similarity_adj[track_index, connected_track_index] = .95


# KGAT Model Implementation: PyTorch

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import math

class KGATLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(KGATLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        #self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.weight = nn.Parameter(torch.randn(in_features, out_features) * 0.1)  # multiply by 0.1 to increase values
        self.reset_parameters()

#    def reset_parameters(self):
#        gain = nn.init.calculate_gain('relu') * math.sqrt(3)
#        nn.init.uniform_(self.weight, -10, 10)
#        self.weight.data.mul_(gain)

    def reset_parameters(self):
        #print(self.weight)
        #nn.Parameter(torch.randn(11, out_features) * 0.1)  # multiply by 0.1 to increase values
        nn.init.kaiming_uniform_(self.weight, mode='fan_in', nonlinearity='relu')
        #print(self.weight)

    def forward(self, adjacency_matrix, input_features):
        output_features = torch.mm(input_features, self.weight)
        return torch.mm(adjacency_matrix, output_features)

class KGAT(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, num_layers):
        super(KGAT, self).__init__()
        self.input_bn = nn.BatchNorm1d(hidden_features)
        self.hidden_bns = nn.ModuleList([nn.BatchNorm1d(hidden_features) for _ in range(num_layers - 1)])
        self.in_features = in_features
        self.hidden_features = hidden_features
        self.out_features = out_features
        self.num_layers = num_layers

        # Define the input layer
        self.input_layer = KGATLayer(in_features, hidden_features)

        # Define the hidden layers
        self.hidden_layers = nn.ModuleList()
        for _ in range(num_layers - 1):
            self.hidden_layers.append(KGATLayer(hidden_features, hidden_features))

        # Define the output layer
        self.output_layer = KGATLayer(hidden_features, out_features)

    def forward(self, adjacency_matrix, input_features):
        x = self.input_layer(adjacency_matrix, input_features)
        x = x.unsqueeze(2)  # Add an extra dimension for Batch Normalization
        x = F.relu(self.input_bn(x))
        x = x.squeeze(2)  # Remove the extra dimension
        for i, layer in enumerate(self.hidden_layers):
            x = layer(adjacency_matrix, x)
            x = x.unsqueeze(2)  # Add an extra dimension for Batch Normalization
            x = F.relu(self.hidden_bns[i](x))
            x = x.squeeze(2)  # Remove the extra dimension

        # Pass through the output layer
        x = self.output_layer(adjacency_matrix, x)

        return x        

hidden_features = 64  # Number of hidden features in the KGAT model
out_features = 1  # Number of output features in the KGAT model
num_layers = 2  # Number of layers in the KGAT model

numeric_keys = [
    'acousticness', 'danceability', 'duration_ms', 'energy', 'explicit',
    'liveness', 'loudness', 'popularity', 'speechiness', 'tempo', 'valence'
]

model = KGAT(num_layers=num_layers, in_features=len(numeric_keys), hidden_features=hidden_features, out_features=out_features)



# Prepare Model

In [11]:
import random
from sklearn.model_selection import train_test_split
from tqdm import tqdm

contains_edges_flat = [(playlist_id, track_id) for playlist_id, track_ids in contains_edges.items() for track_id in track_ids]
train_edges, val_edges = train_test_split(contains_edges_flat, test_size=0.2, random_state=42)

from torch.utils.data import Dataset, DataLoader

class MusicDataset(Dataset):
    def __init__(self, edges, playlist_id_to_index, track_id_to_index):
        self.edges = edges
        self.playlist_id_to_index = playlist_id_to_index
        self.track_id_to_index = track_id_to_index

    def __len__(self):
        return len(self.edges)

    def __getitem__(self, idx):
        playlist_id, track_id = self.edges[idx]
        playlist_index = self.playlist_id_to_index[playlist_id]
        track_index = self.track_id_to_index[track_id]
        return playlist_index, track_index

train_dataset = MusicDataset(train_edges, playlist_id_to_index, track_id_to_index)
val_dataset = MusicDataset(val_edges, playlist_id_to_index, track_id_to_index)

batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

loss_function = nn.BCEWithLogitsLoss()  # Binary cross-entropy loss with logits for link prediction
#optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Adam optimizer with a learning rate of 'lr'
optimizer = torch.optim.SGD(model.parameters(), lr=0.0002, momentum=0.90)
num_epochs = 50

# Convert adjacency matrices to PyTorch tensors
shared_album_adj_tensor = torch.FloatTensor(shared_album_adj.toarray())
shared_artist_adj_tensor = torch.FloatTensor(shared_artist_adj.toarray())
shared_genre_adj_tensor = torch.FloatTensor(shared_genre_adj.toarray())
cosine_similarity_adj_tensor = torch.FloatTensor(cosine_similarity_adj.toarray())

# Function to handle tensor conversion for either array or string values
def process_feature(feature):
    if isinstance(feature, (list, tuple, np.ndarray)):
        return np.array(feature, dtype=np.float32)
    else:
        return np.float32(feature)

# Convert track features to PyTorch tensor
track_features = torch.FloatTensor([[process_feature(track_dict[track_id][key]) for key in numeric_keys] for track_id in unique_track_ids])

# Set device to use
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move model and data to the selected device
model = model.to(device)
shared_album_adj_tensor = shared_album_adj_tensor.to(device)
shared_artist_adj_tensor = shared_artist_adj_tensor.to(device)
shared_genre_adj_tensor = shared_genre_adj_tensor.to(device)
cosine_similarity_adj_tensor = cosine_similarity_adj_tensor.to(device)
track_features = track_features.to(device)

# Train KGAT Model

vanishing gradient problem in training this model.

Weights are initialized, but after passing each batch through the training loop the weights are equal to zero for the input layer, hidden layer, and output layers.

In [6]:
# Training loop
for epoch in tqdm(range(num_epochs)):
    model.train()
    total_loss = 0
    
    count = 0
    
    for batch_idx, (playlist_indices, track_indices) in enumerate(train_loader):
        playlist_indices = playlist_indices.to(device)
        track_indices = track_indices.to(device)

        optimizer.zero_grad()
        
        # Obtain embeddings from the KGAT model
        track_embeddings_shared_album = model(shared_album_adj_tensor, track_features)
        track_embeddings_shared_artist = model(shared_artist_adj_tensor, track_features)
        track_embeddings_shared_genre = model(shared_genre_adj_tensor, track_features)
        track_embeddings_cosine_similarity = model(cosine_similarity_adj_tensor, track_features)

        # Combine the embeddings from different relationships
        track_embeddings = track_embeddings_shared_album + track_embeddings_shared_artist + track_embeddings_shared_genre + track_embeddings_cosine_similarity

        # Compute the predicted probability of a track belonging to a playlist
        predicted_probs = torch.sigmoid(torch.sum(track_embeddings[track_indices] * track_embeddings[playlist_indices], dim=1))

        # Compute the ground truth for the current batch
        ground_truth = torch.tensor([1.0] * len(track_indices), device=device)

        # Compute the loss
        loss = loss_function(predicted_probs, ground_truth)

        # Perform backpropagation and optimization
        loss.backward()
        
        if ((count % 100) == 0) or ((count % 100) == 1) or ((count % 100) == 2):
            for name, param in model.named_parameters():
                if param.requires_grad and param.grad is not None:
                    print(f'Epoch [{epoch + 1}/{num_epochs}], Batch [{batch_idx + 1}/{len(train_loader)}], Gradient of {name}: {param.grad.mean()}')
        count += 1
        
        optimizer.step()

        total_loss += loss.item()

    # Compute the average loss for this epoch
    avg_loss = total_loss / len(train_loader)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}')

    # Evaluate the model on the validation set
    model.eval()
    val_loss = 0

    with torch.no_grad():
        for playlist_indices, track_indices in val_loader:
            playlist_indices = playlist_indices.to(device)
            track_indices = track_indices.to(device)

            # Obtain embeddings from the KGAT model
            track_embeddings_shared_album = model(shared_album_adj_tensor, track_features)
            track_embeddings_shared_artist = model(shared_artist_adj_tensor, track_features)
            track_embeddings_shared_genre = model(shared_genre_adj_tensor, track_features)
            track_embeddings_cosine_similarity = model(cosine_similarity_adj_tensor, track_features)

            # Combine the embeddings from different relationships
            track_embeddings = track_embeddings_shared_album + track_embeddings_shared_artist + track_embeddings_shared_genre + track_embeddings_cosine_similarity

            # Compute the predicted probability of a track belonging to a playlist
            predicted_probs = torch.sigmoid(torch.sum(track_embeddings[track_indices] * track_embeddings[playlist_indices], dim=1))

            # Compute the ground truth for the current batch
            ground_truth = torch.tensor([1.0] * len(track_indices), device=device)

            # Compute the loss
            loss = loss_function(predicted_probs, ground_truth)

            val_loss += loss.item()

    # Compute the average validation loss for this epoch
    avg_val_loss = val_loss / len(val_loader)
    print(f'Validation Loss: {avg_val_loss:.4f}')


  0%|                                                    | 0/50 [00:00<?, ?it/s]

Epoch [1/50], Batch [1/184], Gradient of input_bn.weight: -3.808236215263605e-08
Epoch [1/50], Batch [1/184], Gradient of input_bn.bias: -0.000841515779029578
Epoch [1/50], Batch [1/184], Gradient of hidden_bns.0.weight: 0.00025379256112501025
Epoch [1/50], Batch [1/184], Gradient of hidden_bns.0.bias: 0.0005878241499885917
Epoch [1/50], Batch [1/184], Gradient of input_layer.weight: 8.912380508263595e-06
Epoch [1/50], Batch [1/184], Gradient of hidden_layers.0.weight: -0.0007725466275587678
Epoch [1/50], Batch [1/184], Gradient of output_layer.weight: 0.0027055959217250347
Epoch [1/50], Batch [2/184], Gradient of input_bn.weight: -1.4295801520347595e-07
Epoch [1/50], Batch [2/184], Gradient of input_bn.bias: -0.00036191369872540236
Epoch [1/50], Batch [2/184], Gradient of hidden_bns.0.weight: 0.0002734675072133541
Epoch [1/50], Batch [2/184], Gradient of hidden_bns.0.bias: 0.0007899461779743433
Epoch [1/50], Batch [2/184], Gradient of input_layer.weight: -1.3507545190805104e-05
Epoch 

  2%|▊                                        | 1/50 [09:24<7:41:05, 564.59s/it]

Validation Loss: 0.3339
Epoch [2/50], Batch [1/184], Gradient of input_bn.weight: 1.3348221727937926e-06
Epoch [2/50], Batch [1/184], Gradient of input_bn.bias: -1.364789204671979e-05
Epoch [2/50], Batch [1/184], Gradient of hidden_bns.0.weight: 4.159519448876381e-05
Epoch [2/50], Batch [1/184], Gradient of hidden_bns.0.bias: -0.00016994297038763762
Epoch [2/50], Batch [1/184], Gradient of input_layer.weight: 4.793313337358995e-07
Epoch [2/50], Batch [1/184], Gradient of hidden_layers.0.weight: -3.384886804269627e-05
Epoch [2/50], Batch [1/184], Gradient of output_layer.weight: 0.0012177529279142618
Epoch [2/50], Batch [2/184], Gradient of input_bn.weight: 7.86027612775797e-07
Epoch [2/50], Batch [2/184], Gradient of input_bn.bias: -5.620004958473146e-05
Epoch [2/50], Batch [2/184], Gradient of hidden_bns.0.weight: 3.4782078728312626e-05
Epoch [2/50], Batch [2/184], Gradient of hidden_bns.0.bias: 0.00019568981952033937
Epoch [2/50], Batch [2/184], Gradient of input_layer.weight: -1.852

  4%|█▋                                       | 2/50 [18:47<7:30:41, 563.37s/it]

...

Validation Loss: 0.3294
Epoch [49/50], Batch [1/184], Gradient of input_bn.weight: 2.3490883904742077e-07
Epoch [49/50], Batch [1/184], Gradient of input_bn.bias: 7.15633041181718e-06
Epoch [49/50], Batch [1/184], Gradient of hidden_bns.0.weight: -6.812336323491763e-06
Epoch [49/50], Batch [1/184], Gradient of hidden_bns.0.bias: -4.684927625930868e-05
Epoch [49/50], Batch [1/184], Gradient of input_layer.weight: 5.6615679255855866e-08
Epoch [49/50], Batch [1/184], Gradient of hidden_layers.0.weight: -1.0200595170317683e-05
Epoch [49/50], Batch [1/184], Gradient of output_layer.weight: 7.912825094535947e-05
Epoch [49/50], Batch [2/184], Gradient of input_bn.weight: 1.5356317817349918e-07
Epoch [49/50], Batch [2/184], Gradient of input_bn.bias: 4.034543508169008e-06
Epoch [49/50], Batch [2/184], Gradient of hidden_bns.0.weight: -5.289507498673629e-09
Epoch [49/50], Batch [2/184], Gradient of hidden_bns.0.bias: -7.917155016912147e-06
Epoch [49/50], Batch [2/184], Gradient of input_layer.w

 98%|███████████████████████████████████████▏| 49/50 [7:40:13<09:23, 563.50s/it]

Validation Loss: 0.3293
Epoch [50/50], Batch [1/184], Gradient of input_bn.weight: 3.0268893169704825e-08
Epoch [50/50], Batch [1/184], Gradient of input_bn.bias: 3.837144504359458e-06
Epoch [50/50], Batch [1/184], Gradient of hidden_bns.0.weight: -1.927667199197458e-07
Epoch [50/50], Batch [1/184], Gradient of hidden_bns.0.bias: 1.3225001566752326e-05
Epoch [50/50], Batch [1/184], Gradient of input_layer.weight: -1.3810705468131346e-08
Epoch [50/50], Batch [1/184], Gradient of hidden_layers.0.weight: -2.9994362193974666e-05
Epoch [50/50], Batch [1/184], Gradient of output_layer.weight: 4.059905404574238e-05
Epoch [50/50], Batch [2/184], Gradient of input_bn.weight: 9.573386705596931e-08
Epoch [50/50], Batch [2/184], Gradient of input_bn.bias: 3.623121529017226e-06
Epoch [50/50], Batch [2/184], Gradient of hidden_bns.0.weight: 2.104438408423448e-06
Epoch [50/50], Batch [2/184], Gradient of hidden_bns.0.bias: 1.4189294233801775e-05
Epoch [50/50], Batch [2/184], Gradient of input_layer.w

100%|████████████████████████████████████████| 50/50 [7:49:37<00:00, 563.55s/it]

Validation Loss: 0.3293


In [7]:
torch.save(model.state_dict(), 'trained_kgat_model_v100.pth')


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import math

class KGATLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(KGATLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        #self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.weight = nn.Parameter(torch.randn(in_features, out_features) * 0.1)  # multiply by 0.1 to increase values
        self.reset_parameters()

#    def reset_parameters(self):
#        gain = nn.init.calculate_gain('relu') * math.sqrt(3)
#        nn.init.uniform_(self.weight, -10, 10)
#        self.weight.data.mul_(gain)

    def reset_parameters(self):
        #print(self.weight)
        #nn.Parameter(torch.randn(11, out_features) * 0.1)  # multiply by 0.1 to increase values
        nn.init.kaiming_uniform_(self.weight, mode='fan_in', nonlinearity='relu')
        #print(self.weight)

    def forward(self, adjacency_matrix, input_features):
        output_features = torch.mm(input_features, self.weight)
        return torch.mm(adjacency_matrix, output_features)

class KGAT(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, num_layers):
        super(KGAT, self).__init__()
        self.input_bn = nn.BatchNorm1d(hidden_features)
        self.hidden_bns = nn.ModuleList([nn.BatchNorm1d(hidden_features) for _ in range(num_layers - 1)])
        self.in_features = in_features
        self.hidden_features = hidden_features
        self.out_features = out_features
        self.num_layers = num_layers

        # Define the input layer
        self.input_layer = KGATLayer(in_features, hidden_features)

        # Define the hidden layers
        self.hidden_layers = nn.ModuleList()
        for _ in range(num_layers - 1):
            self.hidden_layers.append(KGATLayer(hidden_features, hidden_features))

        # Define the output layer
        self.output_layer = KGATLayer(hidden_features, out_features)

    def forward(self, adjacency_matrix, input_features):
        x = self.input_layer(adjacency_matrix, input_features)
        x = x.unsqueeze(2)  # Add an extra dimension for Batch Normalization
        x = F.relu(self.input_bn(x))
        x = x.squeeze(2)  # Remove the extra dimension
        for i, layer in enumerate(self.hidden_layers):
            x = layer(adjacency_matrix, x)
            x = x.unsqueeze(2)  # Add an extra dimension for Batch Normalization
            x = F.relu(self.hidden_bns[i](x))
            x = x.squeeze(2)  # Remove the extra dimension

        # Pass through the output layer
        x = self.output_layer(adjacency_matrix, x)

        return x        

hidden_features = 64  # Number of hidden features in the KGAT model
out_features = 1  # Number of output features in the KGAT model
num_layers = 2  # Number of layers in the KGAT model

numeric_keys = [
    'acousticness', 'danceability', 'duration_ms', 'energy', 'explicit',
    'liveness', 'loudness', 'popularity', 'speechiness', 'tempo', 'valence'
]

model = KGAT(num_layers=num_layers, in_features=len(numeric_keys), hidden_features=hidden_features, out_features=out_features)

In [4]:
loaded_model = KGAT(num_layers=num_layers, in_features=len(numeric_keys), hidden_features=hidden_features, out_features=out_features)
loaded_model.load_state_dict(torch.load('trained_kgat_model_v100.pth'))


<All keys matched successfully>

In [5]:
from pymongo import MongoClient

# connect to MongoDB
username = "..."
password = "..."
cluster_name = "..."
dbname = "..."
client = MongoClient(f"mongodb+srv://{username}:{password}@{cluster_name}.mongodb.net/{dbname}?retryWrites=true&w=majority")
db = client[dbname]
collection = db["tune-users"]

# define the query
query = { "user_id": "..." }

result = collection.find(query, { "top_tracks": 1, "_id": 0 })

track_ids = []

for document in result:
    for track in document['top_tracks']:
        track_ids.append(track['id'])

In [18]:
required_features = [
    't.acousticness', 't.danceability', 't.duration_ms', 't.energy', 't.explicit',
    't.liveness', 't.loudness', 't.popularity', 't.speechiness', 't.tempo', 't.valence'
]
track_features = []
for track in track_result:
    x_features = [track[feature] for feature in required_features]
    track_features.append(x_features)
track_features_tensor = torch.FloatTensor(track_features)

In [19]:
cosine_similarity_adj_tensor = torch.FloatTensor(cosine_similarity_adj.toarray())
def get_ranked_recommendations(user_songs, model, unique_track_ids, track_features):
    # Move the model to the evaluation mode
    model.eval()

    # Create an input tensor containing user's input songs' features
    user_song_indices = [unique_track_ids.index(track_id) for track_id in user_songs if track_id in unique_track_ids]
    user_song_features = track_features[user_song_indices]

    # Calculate the embeddings of the user's input songs using the trained model
    user_song_embeddings = model(cosine_similarity_adj_tensor[:len(user_song_indices), :len(user_song_indices)], user_song_features)
    user_song_embeddings += model(shared_album_adj_tensor[:len(user_song_indices), :len(user_song_indices)], user_song_features)
    user_song_embeddings += model(shared_artist_adj_tensor[:len(user_song_indices), :len(user_song_indices)], user_song_features)
    user_song_embeddings += model(shared_genre_adj_tensor[:len(user_song_indices), :len(user_song_indices)], user_song_features)
    
    # Calculate the average of the input songs' embeddings
    avg_embedding = torch.mean(user_song_embeddings, dim=0, keepdim=True)

    # Calculate the similarity score between the average embedding and all tracks in the dataset
    similarity_scores = torch.mm(track_embeddings, avg_embedding.t()).squeeze()

    # Sort similarity scores in descending order and get the corresponding indices
    sorted_indices = torch.argsort(similarity_scores, descending=True)

    # Convert the sorted indices to track IDs
    ranked_track_ids = [unique_track_ids[idx] for idx in sorted_indices.tolist()]

    # Remove the user's input songs from the ranked recommendations
    recommendations = [track_id for track_id in ranked_track_ids if track_id not in user_songs]

    return recommendations


In [21]:
# Convert adjacency matrices to PyTorch tensors
shared_album_adj_tensor = torch.FloatTensor(shared_album_adj.toarray())
shared_artist_adj_tensor = torch.FloatTensor(shared_artist_adj.toarray())
shared_genre_adj_tensor = torch.FloatTensor(shared_genre_adj.toarray())
cosine_similarity_adj_tensor = torch.FloatTensor(cosine_similarity_adj.toarray())

# Obtain embeddings from the KGAT model
track_embeddings_shared_album = model(shared_album_adj_tensor, track_features_tensor)
track_embeddings_shared_artist = model(shared_artist_adj_tensor, track_features_tensor)
track_embeddings_shared_genre = model(shared_genre_adj_tensor, track_features_tensor)
track_embeddings_cosine_similarity = model(cosine_similarity_adj_tensor, track_features_tensor)

# Combine the embeddings from different relationships
track_embeddings = track_embeddings_shared_album + track_embeddings_shared_artist + track_embeddings_shared_genre + track_embeddings_cosine_similarity

In [22]:
recommendations = get_ranked_recommendations(track_ids, model, unique_track_ids, track_features_tensor)

In [23]:
recommendations[:50]

['1yRl361oS9xXUIDN3GF7pd',
 '19dSGEa2qpyGM60nxoLv3q',
 '2qywCWdMFK2ew9ALHA4UfD',
 '4lIIOHPXeh74fbPUV74zRn',
 '7pN1FnWdKOx3GAciqfVVf7',
 '1Eq8p4OHjhVrSW0Wa815B1',
 '1Q5cjSmt4WTJyJ0EgAFfgg',
 '4o4nIPbrEAoCAq3zqWVA4E',
 '3msjyQTZr7gHspgJCzl4v5',
 '1HW3I6KUzjYPEMqZrlFN66',
 '20uX9Nz9dJBKD7MstFDFs0',
 '1ah3n5wpOQhFQzl9t3qGIu',
 '0gIsM9tn5pP1jnTSrkdHCL',
 '7f32ap55N5PhZBMEr8TnKV',
 '6rOVIb2PyUMBw8b6hUdxHj',
 '2eh3gUtf5GwwlEHPQOgTmV',
 '0OdX5KLktgTp3FAmUqWvLt',
 '2AYtjqogao6Fj3N7cx39Of',
 '0xuvCPMUrbr6xk3myTCqwF',
 '59YiIzi4C1KJwClBBRFucR',
 '4nwwj1bE8KHaBixtEtMu4d',
 '4uew5SER845c4iNj4sl8jG',
 '5deBKQ6qZaTHBRtRNZtL9X',
 '22sdeHdy5DcCVV7dJw4uRn',
 '5gAPTm517kIXqq9YUBLehU',
 '5O71be8zUY85FRBQJsQRWd',
 '2k4O9OqM4RmuGwOgrU272J',
 '4VRcCHaxMHugywsyc33xZa',
 '6JvK5IGgZ1rmenDKHufzIx',
 '0hESC8513XQKkf82EMXpiI',
 '2gc6W4zP2EYcimOCg0tg2v',
 '5eYO9lIshL5wbYDO0WWbbX',
 '2eIAizioh2hRmYRagndFb8',
 '4VfdRKABHsrA0ii7wGDjJm',
 '7uiJYByjnDGY8Z1sAsCSFC',
 '17uRlIf3SeM3VMptNbsZtP',
 '37oH03cQRwoyBUTyGDyixV',
 

In [17]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy

client_credentials_manager = SpotifyClientCredentials(client_id='...', client_secret="...")
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def get_track_info(track_id):
    track_info = sp.track(track_id)
    track_name = track_info['name']
    artist_info = track_info['artists']
    artist_names = [artist['name'] for artist in artist_info]
    return track_name, artist_names

for track_id in recommendations[:50]:
    track_name, artist_names = get_track_info(track_id)
    print(f'Track name: {track_name}')
    print(f'Artist(s): {", ".join(artist_names)}')
    print()

Track name: Euryanthe, J. 291: Overture
Artist(s): Carl Maria von Weber, Tapiola Sinfonietta, Jean-Jacques Kantorow

Track name: Der Freischütz, J. 277 / Act III: "Und ob die Wolke sie verhülle"
Artist(s): Carl Maria von Weber, Gundula Janowitz, Staatskapelle Dresden, Carlos Kleiber

Track name: St Matthew Passion: Iii. Psalm
Artist(s): Bent Sørensen, The Norwegian Soloists' Choir, Grete Pedersen, Ensemble Allegria

Track name: Drei Romanzen, Op. 94: III. Nicht schnell
Artist(s): Robert Schumann, Kim Dami

Track name: Les berceaux, Op. 23, No. 1
Artist(s): Gabriel Fauré, Mischa Maisky, Daria Hovora

Track name: Les Pêcheurs de perles: Je crois entendre encore (Nadir)
Artist(s): Georges Bizet, Nicolai Gedda, Pierre Dervaux, Orchestre du Theatre National de IOpera-Comique, Orchestre Du Theatre National De L'opera-comique

Track name: Il Signor Bruschino: No. 1: Introduzione - Deh tu m’assisti amore
Artist(s): Gioachino Rossini, Massimiliano Barbolini, Alessandro Codeluppi, Clara Giangasp